# Scraping du site beerwulf pour enrichir la base de données

- Nous allons scrapper les données relatives aux bières sur le site www.beerwulf.com. L'avantage de ce site est qu'il dispose de données précises sur les bières : des données physico chimiques, mais aussi des indicateurs subjectifs exhaustifs (couleur de la bière, son goût, ou encore sa longueur). Nous utilisons le site https://www.beerwulf.com/fr-be/c/bieres?page=1&container=Bouteille,Canette pour afficher toutes les bières ainsi que les canettes proposées.
- Avant toute chose, il est nécessaire d'explorer la page https://www.beerwulf.com/robots.txt afin de s'assurer que le web-scraping des données des bières est autorisé. **Ici, il se trouve que c'est le cas.**

On peut alors commencer à importer les librairies nécessaires au scraping.

In [22]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import urllib
from urllib.request import urlopen
import re
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go




options = Options()
options.headless = True # On "n'affiche pas" la recherche internet
options.add_argument("--window-size=1920,1200")
DRIVER_PATH = '/Applications/chromedriver'

### Étape 1 : obtenir de l'url de toutes les bières du site de www.berewulf.com
Le catalogue comporte 25 pages de bières. On peut donc automatiser la collecte des urls grâce à la librairie `selenium` puisque les pages sont codées en Javascript. Bs4 ne peut pas aller scrapper directement les données recherchées. Par la suite, nous utilisons le moteur de recherche de Google Chrome.

In [2]:
# Boucle qui parcourt toutes les pages du site et va collecter le lien `href` de toutes les bières.
nb_pages = 25
liste_urls = []

for i in range(nb_pages):
    driver = webdriver.Chrome(options=options, executable_path=DRIVER_PATH)
    driver.get('https://www.beerwulf.com/fr-be/c?page='+str(i+1)+'&container=Bouteille,Canette&segment=Toutes%20les%20bi%C3%A8res&routeQuery=c')
    elems = driver.find_elements_by_xpath("//div[@id='product-items-container']/a")
    for elem in elems:
        liste_urls.append(elem.get_attribute('href'))

#Vérification
print('Il y a '+str(len(liste_urls))+' urls collectés.')

Il y a 1164 urls collectés.


### Étape 2 : obtenir les caractéristiques de toutes les bières du site, et les stocker dans un dictionnaire
Nous disposons donc de l'url de toutes les bières, donc nous pouvons nous rendre sur ces pages internet grâce à `selenium`. Nous pouvons à présent scrapper leurs caractéristiques individuelles.

Pour ce faire, nous avons d'abord besoin de définir des fonctions qui permettront d'aller scraper les caractéristiques des bières. Les données ne sont pas déjà bien rangées dans un tableau, il faut aller les scrapper individuellement grâce à leur **"Xpath"**, c'est-à-dire le chemin qui permettra à `selenium` d'accéder à la donnée si ce n'est pas possible avec `BeautifulSoup`.

In [3]:
from selenium.common.exceptions import NoSuchElementException, StaleElementReferenceException

def valeur_text(xpath):
    """
    Cette fonction scrappe des caractéristiques de la bière (degré d'alccolémie, nom de 
    la brasserie, etc) au format text si cette dernière est présente sur la page. 
    """
    try : 
        return driver.find_element_by_xpath(xpath).text
    except (NoSuchElementException, StaleElementReferenceException) as e:
        return float('nan')

def valeur_percent(xpath):
    """
    Cette fonction scrappe des caractéristiques de la bière exprimées en pourcentage
    (acidité, amertume, etc) si cette dernière est présente sur la page. 
    """
    try : 
        return driver.find_element_by_xpath(xpath).get_attribute('data-percent')
    except (NoSuchElementException, StaleElementReferenceException) as e:
        return float('nan')


# Scrapper l'IBU d'une bière ne peut être entièrement réalisé avec selenium, il faut donc utiliser bs4
def get_ibu(url):
    """
    url : url de la bière selectionnee
    Fonction renvoie l'IBU de la bière grâce à la librairie BeautifulSoup
    """
    request = urllib.request.Request(url, headers = {'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_9_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/35.0.1916.47 Safari/537.36'})
    html = urllib.request.urlopen(request)
    html_soup = BeautifulSoup(html, 'html.parser')
    try:
        return html_soup.find_all('strong')[0].text
    except IndexError:
        return float('nan')


On peut désormais parcourir les pages des différentes bières de www.beerwulf.com et stocker leurs caractéristiques dans un dictionnaire.

In [4]:
dico_all_beers = {}

for i in range(len(liste_urls)):
    driver.get(liste_urls[i])

    beer_name = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[1]/h1")
    beer_style = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[1]/a")
    country = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[4]")
    brewery_name = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[5]/a")
    bottle = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[2]")
    abv = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[3]")
    ibu = get_ibu(liste_urls[i])
    intensite = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[1]/dd/div")
    beer_type = valeur_percent("/html/body/div[1]/div/div[2]/div/div[2]/div/div[2]/dl/dd[1]/a")
    longueur = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[3]/dd/div")
    acidite = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[4]/dd/div")
    amertume = valeur_percent("/html/body/div[1]/div/div[3]/div/div/div/div/div/div[3]/div[2]/div/div[1]/div/div[5]/dd/div")
    price = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[3]/div[2]/div/span")
    note = valeur_text("/html/body/div[1]/div/div[2]/div/div[2]/div/div[1]/div[3]/div[1]/span")

    liste_caractéristiques = [beer_name, beer_style, country, brewery_name, bottle, abv, ibu, intensite, longueur, acidite, amertume, price, note]
        
    dico_all_beers[beer_name] = liste_caractéristiques

# Affichage des 5 premières bières scrapées
print(list(dico_all_beers.items())[:5])
print('\nNous avons scrapé '+str(len(dico_all_beers))+' bières.')

[('Walhalla Aphrodite Raspberry Berliner Weisse', ['Walhalla Aphrodite Raspberry Berliner Weisse', 'Bière Sour', 'Pays-Bas', 'Walhalla Craft Beer', '33 cl', '4,0%', '-', '0', '40', '40', '0', nan, nan]), ('Affligem Blond 0.0%', ['Affligem Blond 0.0%', 'Bière Blonde', 'Belgique', 'Affligem', '30 cl', '0,0%', nan, nan, nan, nan, nan, '€ 1,59', '(2,84)']), ('Affligem Blond', ['Affligem Blond', 'Bière Blonde', 'Belgique', 'Affligem', '30 cl', '6,8%', '-', '60', '60', '0', '40', nan, '(3,76)']), ('Affligem Tripel', ['Affligem Tripel', 'Bière Triple', 'Belgique', 'Affligem', '30 cl', '9,0%', '-', '60', '60', '20', '60', '€ 1,79', '(3,57)']), ('Ardwen Woinic Rouge', ['Ardwen Woinic Rouge', 'Bière Fruitée', 'France', 'Ardwen', '33 cl', '8,0%', '-', '60', '100', '40', '20', '€ 2,99', '(3,07)'])]

Nous avons scrapé 1145 bières.


**Bilan** : 1164 urls scrappés ont permis de collecter les données de 1145 bières. Il n'y a pas autant de bières que d'urls en raison de mises à jour du site www.beerwulf.com : par exemple, des promotions ponctuelles pour des bières changent la configuration de la page, donc le Xpath menant à la donnée n'est plus le bon...

### Étape 3 : on convertit ensuite notre dictionnaire en dataframe

In [59]:
df_beers = pd.DataFrame.from_dict(dico_all_beers, orient='index') #faire une table à partir d'un dictionnaire
df_beers.columns = ['beer_name', 'beer_style', 'country', 'brewery_name', 'bottle', 'abv', 'ibu', 'intensite', 'longueur', 'acidite', 'amertume', 'price', 'note']
df_beers = df_beers.reset_index(drop=True)
df_beers.head(5)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33 cl,"4,0%",-,0,40,40,0,NaN,NaN
1,Affligem Blond 0.0%,Bière Blonde,Belgique,Affligem,30 cl,"0,0%",NaN,NaN,NaN,NaN,NaN,"€ 1,59","(2,84)"
2,Affligem Blond,Bière Blonde,Belgique,Affligem,30 cl,"6,8%",-,60,60,0,40,NaN,"(3,76)"
3,Affligem Tripel,Bière Triple,Belgique,Affligem,30 cl,"9,0%",-,60,60,20,60,"€ 1,79","(3,57)"
4,Ardwen Woinic Rouge,Bière Fruitée,France,Ardwen,33 cl,"8,0%",-,60,100,40,20,"€ 2,99","(3,07)"


### Étape 4 : on nettoie les données pour leur exploitation future 

On convertir les chaines de caractères en `float` pour plus tard traiter les données

In [60]:
# Fonction qui nettoie les données scrapées au format text. Cela permet ensuite de convertir les données chiffrées en float 
def clean(text):
    text = text.replace(' cl','').replace('%','').replace('(','').replace(')','').replace('€ ','').replace(',','.')
    return text

# On nettoie les entrées du dataframe
for i in range(len(df_beers)):
    for j in ['bottle', 'abv', 'intensite', 'longueur', 'acidite', 'amertume', 'ibu', 'price', 'note']:
        if df_beers[j][i] == '-': # Seules les données relatives à 'ibu' ont des valeurs '-'
            df_beers[j][i] = np.nan
        elif type(df_beers[j][i]) == str and len(df_beers[j][i]) > 6: # Nettoyage des données non chiffrées
            df_beers[j][i] = np.nan
        elif type(df_beers[j][i]) !=  float: 
            df_beers[j][i] = float(clean(df_beers[j][i])) # Nettoyage pour ensuite convertir au format float

df_beers.head(5)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33,4,NaN,0,40,40,0,NaN,NaN
1,Affligem Blond 0.0%,Bière Blonde,Belgique,Affligem,30,0,NaN,NaN,NaN,NaN,NaN,1.59,2.84
2,Affligem Blond,Bière Blonde,Belgique,Affligem,30,6.8,NaN,60,60,0,40,NaN,3.76
3,Affligem Tripel,Bière Triple,Belgique,Affligem,30,9,NaN,60,60,20,60,1.79,3.57
4,Ardwen Woinic Rouge,Bière Fruitée,France,Ardwen,33,8,NaN,60,100,40,20,2.99,3.07


### Nettoyage dataframe pour ne garder que les bières alcoolisées

Avant de réaliser des statistiques descriptives sur la base de données scrappées, on regarde s'il y a des bières sans alcool. Le cas échéant, on les supprime du dataset car notre analyse se concentre uniquement sur les boissons alcoolisées.

In [63]:
bieres_sans_alcool = df_beers[df_beers['abv'].isin([0,np.nan])] 
bieres_sans_alcool.head()

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note
1,Affligem Blond 0.0%,Bière Blonde,Belgique,Affligem,30,0,NaN,NaN,NaN,NaN,NaN,1.59,2.84
51,Heineken 0.0,Lager,Pays-Bas,Heineken Brewery,30,0,16,40,40,40,40,1.49,2.3
162,"Budels Malty Dark 0,0%",Bière Bock,Pays-Bas,Budelse Brouwerij,30,0,NaN,40,60,20,20,1.79,1.85
180,Brand Weizen 0.0,Bière Weiss,Pays-Bas,Brand,30,0,12,60,60,20,20,1.79,2.96
183,Palm 0.0,Pale Ale,Belgique,Brouwerij Palm,25,0,NaN,60,60,20,40,1.89,2.06
217,Braxzz Porter,Bière Porter & Stout,Pays-Bas,NaN,33,0,50,60,80,0,40,2.49,2.51
352,Brand IPA 0.0%,IPA,Pays-Bas,Brand,30,0,NaN,60,0,0,0,1.89,2.96
353,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


On remarque que nous en avons scrappé quelques bières sans alcool. D'ailleurs, ces dernières n'ont pas de "bonne" note. On les supprime du dataframe.

In [64]:
df_beers = df_beers.drop([df_beers[df_beers['abv'].isin([0,np.nan])].index[i] for i in range(len(df_beers[df_beers['abv'].isin([0,np.nan])]))]).reset_index(drop = True)

### Encodage du prix de la pinte

Les prix ne peuvent être directement comparés dans la mesure où le volume de la bouteille vendue diffère souvent. Nous décidons d'ajouter une colonne `prix_de_la_pinte` égale au prix de la pinte de cette bière pour pouvoir comparer ces données ensuite. 

In [66]:
df_beers['prix_de_la_pinte'] = 0.0

for i in range(len(df_beers)):
    if df_beers['price'][i] != np.nan:
        df_beers['prix_de_la_pinte'][i] = round(50*df_beers['price'][i]/df_beers['bottle'][i],2)
    else:
        df_beers['prix_de_la_pinte'][i] = np.nan

df_beers.head(3)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33,4,NaN,0,40,40,0,NaN,NaN,NaN
1,Affligem Blond,Bière Blonde,Belgique,Affligem,30,6.8,NaN,60,60,0,40,NaN,3.76,NaN
2,Affligem Tripel,Bière Triple,Belgique,Affligem,30,9,NaN,60,60,20,60,1.79,3.57,2.98


### Formatage des données : convertir les données relatives à `intensite`, `longueur`, `acidite`, `amertume` en une note sur 5 pour pouvoir ensuite faire le lien avec l'autre base de données.
Les notes des bières concernant l'`intensite`, la `longueur` et l'`amertume` vont de 0 à 100%, tandis que la note de l'acidité varie entre 0 et 80%. On adapte donc le formatage.

In [67]:
for i in range(len(df_beers)):
    for j in ['intensite', 'longueur','amertume']:
        if df_beers[j][i] != np.nan:
            df_beers[j][i] = df_beers[j][i]/20
    if df_beers['acidite'][i] != np.nan:
        df_beers['acidite'][i] = df_beers['acidite'][i]/16

df_beers.head(10)

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
0,Walhalla Aphrodite Raspberry Berliner Weisse,Bière Sour,Pays-Bas,Walhalla Craft Beer,33,4,NaN,0,2,2.5,0,NaN,NaN,NaN
1,Affligem Blond,Bière Blonde,Belgique,Affligem,30,6.8,NaN,3,3,0,2,NaN,3.76,NaN
2,Affligem Tripel,Bière Triple,Belgique,Affligem,30,9,NaN,3,3,1.25,3,1.79,3.57,2.98
3,Ardwen Woinic Rouge,Bière Fruitée,France,Ardwen,33,8,NaN,3,5,2.5,1,2.99,3.07,4.53
4,Atlantic Blanche des Gabariers,Bière Blanche,France,Brasserie des Gabariers,33,5,NaN,2,3,1.25,1,2.79,2.93,4.23
5,Asahi Super Dry,Lager,Japon,NaN,33,5.2,16,3,2,0,2,2.39,3.07,3.62
6,Atlantic Rubis des Gabariers,Bière Ambrée,France,Brasserie des Gabariers,33,6,NaN,2,4,3.75,1,2.99,2.79,4.53
7,Atlantic Dorée des Gabariers,Bière Ambrée,France,Brasserie des Gabariers,33,5.5,NaN,2,3,1.25,2,2.79,2.84,4.23
8,Bacchus Framboos,Bière Fruitée,Belgique,Brouwerij Van Honsebrouck,38,5,NaN,4,3,3.75,1,3.49,3.6,4.59
9,Baladin Nora,Bière Blonde,Italie,Baladin,33,6.8,11,3,3,0,1,3.49,3.63,5.29


In [130]:
fig = go.Figure()

fig.add_trace(go.Box(y=df_beers['intensite'], name = 'Intensité'))
fig.add_trace(go.Box(y=df_beers['longueur'], name = 'Longueur'))
fig.add_trace(go.Box(y=df_beers['amertume'], name = 'Amertume'))
fig.add_trace(go.Box(y=df_beers['acidite'], name = 'Acidité'))

fig.layout.update(yaxis_title="Note", title="Notes des caractéristiques physico-chimiques", width = 800, title_x=0.5)
fig.show()

Les boîtes à moustaches révèlent que :
- Le 1er quartile et le 3ème quartile concernant l'intensité des bières sont de 3 sur 5. En d'autres termes, plus de la moitié des bières ont donc cette même intensité et les notes extrèmes proches de 0 sur 5 ou de 5 sur 5 sont minoritaires.
- La longueur et l'amertume des bières scrappées sont majoritairement comprises entre 2 et 3, ce qui est la norme dans le domaine des bières. 
- Les bières scrappées sont peu acides, ce qui semble logique dans la mesure où le dataset comporte des données de bières disponibles à l'achat. Autant qu'elles ne soient pas trop acides !

En un mot, les caractéristiques physico-chimiques scrappées semblent cohérentes.

## Statistiques descriptives de la base de données scrappées

Les données étant désormais prêtes à leur exploitation, nous pouvons réaliser quelques aalyses descriptives afin d'avoir une vision d'ensemble du dataset.

In [16]:
df_beers.isnull().sum()

beer_name             1
beer_style            7
country               0
brewery_name        368
bottle                0
abv                   0
ibu                 485
intensite            62
longueur             62
acidite              62
amertume             62
price               170
note                 79
prix_de_la_pinte    170
dtype: int64

Beaucoup de bières n'ont pas leur ibu renseigné. Cela représente presque 1/3 des données collectées. 

Des bières n'ont pas de donnée relative à leur intensité, à leur longueur et à leur amertume et ce sont probablement les mêmes. Ces données manquantes représentent 5% de toutes les bières scrappées.

Si nous décidons d'éliminer les valeurs manquantes pour les variables qui nous intéressent (ce n'est pas vraiment le cas de `brewery_name` par exemple), nous risquons de perdre presque la moitié des bières scrappées. Ceci est la conséquence du manque de données complètes sur le site www.beerwulf.com.

In [10]:
df_beers.describe()

,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
count,1130.000000,1130.000000,645.000000,1068.000000,1068.000000,1068.000000,1068.000000,960.000000,1051.000000,960.000000
mean,33.269912,6.350088,35.848062,2.976592,2.624532,0.884831,2.251873,2.882219,3.491227,4.332208
std,3.920001,2.074320,20.030920,0.811362,0.821705,1.081587,1.008514,0.692156,0.269224,0.931041
min,25.000000,0.200000,4.000000,0.000000,0.000000,0.000000,0.000000,1.190000,2.000000,1.900000
25%,33.000000,5.000000,22.000000,3.000000,2.000000,0.000000,2.000000,2.490000,3.350000,3.780000
50%,33.000000,6.100000,32.000000,3.000000,3.000000,1.250000,2.000000,2.790000,3.510000,4.230000
75%,33.000000,7.500000,45.000000,3.000000,3.000000,1.250000,3.000000,3.090000,3.660000,4.580000
max,65.000000,16.500000,220.000000,5.000000,5.000000,6.250000,5.000000,7.990000,4.450000,11.800000


Les statistiques de ce tableau semblent cohérentes : 
- Le degré alcoolémique (`abv`)n'excède pas 16.5° et une grande majorité est comprise entre 5° et 8°.
- L'`ibu` très élevé de certaines bières peut intriguer, mais le fin connaisseur saura que de telles bières existent.
- Les notes de `intensite`, `longueur`, `acidite` et de `amertume` s'étalent bien entre 0 et 5. En effet, des bières peuvent se démarquer pour leur forte amertume (ou pas et donc auront la note de 0) ou encore par l'intensité de leurs aromes (ou pas et auront aussi une note de 0).
- La `note` de satisfation de la bière est bien comprise entre 0 et 5 mais son écart-type est très faible. On s'attend plus tard à rencontrer des difficultés pour distinguer les très bonnes bières des bonnes bières.
- Les prix (`price`) semblent honnêtes. Néanmoins, on se doute qu'une bière vendue à 7.99€ est probablement une bière très forte et dont le volume de la bouteille dépasse les 50 cl. Nous allons vérifier cette donnée.

In [14]:
df_beers[df_beers['price'] > 7]

,beer_name,beer_style,country,brewery_name,bottle,abv,ibu,intensite,longueur,acidite,amertume,price,note,prix_de_la_pinte
534,Founders KBS,Bière Porter & Stout,Etats-Unis d'Amérique,NaN,36.0,12.3,70.0,5.0,4.0,0.0,4.0,7.99,4.45,11.1
1019,De Dochter van de Korenaar L'Ensemble di Monta...,Bière Barley Wine,Belgique,NaN,33.0,13.0,NaN,NaN,NaN,NaN,NaN,7.79,4.11,11.8


La bière "Founders KBS" et "De Dochter van de Korenaar L'Ensemble di Montalcino" lèvent nos doutes sur les valeurs maximales de `price` et `prix_de_la_pinte`. Ce sont des bières fortes, d'un style peu commun et qui ont une très bonne note de satisfaction. La Founders KBS est même la bière qui a obtenu la meilleure note globale !

On regarde à présent le `style` des bières scrappées.

In [104]:
fig = px.histogram(
    x=df_beers['beer_style'].value_counts().index, 
    y=df_beers['beer_style'].value_counts(), 
    color = df_beers['beer_style'].value_counts().index,
    title="Répartition des bières scrappées selon leur style")
fig.layout.update(xaxis_title="Style", yaxis_title="Nombre de bières", width = 1000, showlegend=False, title_x=0.5)
fig.show()

On compte 19 styles de bières différents pour ce dataset. C'est une bonne nouvelle car tous les styles de bière sont représentés. Les bières IPAs (179), blondes (147), Pale Ale (105), Porter & Stout (83) et Triple (80) sont un peu plus nombreuses, tandis que les autres styles de bières sont équitablement répartis avec un contingent moyen de 40 bières (hormis pour le cidre qui ne nous intéresse pas).

Même si le style de la bière n'est pas une caractéristique physico-chimique en soi, on peut se demander si cette variable catégorielle pourrait contribuer à mieux prédire la qualité d'une bière. On garde cette idée dans un coin de notre tête.

In [129]:
fig = px.box(df_beers, x="beer_style", y="abv") 
fig.layout.update(xaxis_title="Style", yaxis_title="Degré d'alcoolémie", width = 1000, title="Degré d'alcoolémie en fonction du style de bières", title_x=0.5)

fig.show()

Le degré alcoolémique des bières va dans le sens de nos intuitions : 
- Les bières triples, quadruples, Barley Wine de style Porter & Stout sont "fortes" dans la mesure où plus de la moitié des bières ont un degré d'alcool supérieur à 8°.
- Les autres bières ont un degré d'alccolémie entre 4.5 et 6.5, ce qui confirme le tableau des statistiques descriptives vu précédemment.

## Conclusion

Les bières sont très variées ont peu de valeurs manquantes pour les caractéristiques physico-chimiques. Notre dataset étant prêt à être exploité, on peut l'exporter en tant que fichier csv. 

In [ ]:
df_beers.to_csv('data_beerwulf')